# “Brain Freeze” – Using QPAM to distort incoming neural signals

I took my main inspiration from this research conducted at CCRMA Stanford: https://www.researchgate.net/publication/265165122_Sonification_and_Visualization_of_Neural_Data
https://ccrma.stanford.edu/~mindyc/256a/final/


This project nvestigates the feasibility and implications of Quantum Phase Amplitude Modulation (QPAM) in altering neural transmission. This abstract outlines a theoretical framework where QPAM techniques could intentionally disrupt neural pathways, analogous to inducing temporary cognitive effects such as "brain freeze". Having this happen on trillions of neurons would be quite loud; however here we only use a few artificial neurons. The potential applications and ethical considerations of this approach in neuroscience and cognitive research are explored, suggesting new avenues for understanding neural dynamics and therapeutic interventions and even creating some cool backing sounds!

Since I have familiarity with both Qiskit and Pennylane (Another Quantum Programming Language but using Photonic Qubits) I just decided to use Pennylane due to the fact that in the future I will want to add some QML to automate it. I used Quantum Amplitude Encoding to extract and modify the data. 

In [ ]:
!pip install pennylane

Here we import our Incoming Neural Signals as .wav files.

Here we use real neural signals from a rat brain played by a piano. 

In [ ]:
from IPython.display import Audio
from IPython.display import Image
display(Audio("WakeNeurons.wav")) #Note for this one the array data is const
display(Audio("PoissonWake.wav")) 

And so, to begin making our own audio effect using a quantum computer, we first import required libraries needed and make the function to import .wav audio files

In [ ]:

import pennylane as qml
import pennylane.numpy as np
import scipy.io.wavfile
import wave
import time
import math
import matplotlib.pyplot as plt
import copy


def load_wav(filename):
    sample_rate, data = scipy.io.wavfile.read(filename)
    
    if len(data.shape) == 2:
        data = data.mean(axis=1)
    return sample_rate, data

# save wav
def save_wav(filename, sample_rate, data):
    scipy.io.wavfile.write(filename, sample_rate, data)

In [ ]:
filename = 'PoissonWake.wav'  # encode to quantum computer

In [ ]:
sample_rate, original_data = load_wav(filename)

def QPAM_Encoding_Normalization(samples):
    # Normalize the array between 0 and 2
    normalized_arr = 2 * (samples - np.min(samples)) / (np.max(samples) - np.min(samples))

    # Increase all amplitudes by 1.
    # Halve the adjusted amplitudes.
    # Divide by the total sum of these amplitudes.
    # Calculate the square root of the outcome.

    np_data = np.array(normalized_arr)
    np_data += 1
    np_data /= 2
    sum = np.sum(np_data)
    np_data = np_data/sum

    print(np_data)
    return np_data

features = QPAM_Encoding_Normalization(original_data)
length = features.shape[0]
nb_qubits = math.ceil(np.log2(length))

In [ ]:
#set up the device and num of qubits
dev = qml.device('default.qubit', wires=nb_qubits)
nb_qubits

In [ ]:
@qml.qnode(dev)
def circuit(f=None):
    
    # Embed input features into quantum amplitudes on all qubits
    qml.AmplitudeEmbedding(features=f, wires=range(nb_qubits), pad_with=0,normalize=True)
    
    # Measure the state and return the probabilities of each measurement outcome
    return qml.probs(wires=range(nb_qubits))
results = circuit(features)
print(f"encoding result: {results}")


In [ ]:
normalized_results = ((results - np.min(results)) / (np.max(results) - np.min(results))) * (32767 + 32768) - 32768
newwave_name = f'new_{time.time()}.wav'
quantum_samples_array = []
with wave.open(newwave_name, 'w') as wav_file:
    
    wav_file.setparams((1, 2, sample_rate, length, 'NONE', 'not compressed'))
    
    quantum_samples_array = np.array(normalized_results, dtype=np.int16)
    print(quantum_samples_array)
    wav_file.writeframes(np.array(quantum_samples_array).tobytes())
print(f"{newwave_name} file has been created.")
orig_result = copy.deepcopy(normalized_results)
copy_result = copy.deepcopy(normalized_results)
display(Audio(newwave_name))

Lets see the frequency spectrum and the waveform of the decoded sound !

In [ ]:
sample_rate, data = scipy.io.wavfile.read(newwave_name)
print(sample_rate)
# If stereo, convert to mono by averaging the two channels
if len(data.shape) == 2:
    data = data.mean(axis=1)

# Perform the Fourier Transform to get frequency content
# Use np.fft.rfft to handle real inputs and improve efficiency
freq_data = np.fft.rfft(data)

# Get the power spectrum (magnitude of the Fourier coefficients)
power_spectrum = np.abs(freq_data)

# Generate frequency axis (only for positive frequencies)
freqs = np.fft.rfftfreq(len(data), d=1/sample_rate)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(freqs, power_spectrum)
plt.title(f'Frequency Content of {filename}')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.yscale('log')  
plt.xlim(0, sample_rate / 2)  # Nyquist limit
plt.grid(True)
plt.show()

In [ ]:
#graphing
plt.figure(figsize=(10, 6))
plt.plot(orig_result)
plt.xlim(20000, 30000)
plt.title("Array Data")
plt.xlabel("Index")
plt.ylabel("Value")
plt.grid(True)
plt.show()

Lets see what happen if we "deep freeze" it. That is, since QPAM is probabilistic in nature, we want to see how it might sounds after going through the encoding and decoding phase multiple times. lets test the audio "degradation" after going through the quantum audio encoding and decoding process a couple of time to hear the fidelity. We should hear a clear change compared to the single QPAM result.

In [ ]:
copy_result = copy.deepcopy(orig_result) 
deepfry_quantum_samples_array = copy.deepcopy(quantum_samples_array)
for i in range(2): #BRAINFREEEZE (change the number to make it more chilly!) 
    deepfry_result = circuit(QPAM_Encoding_Normalization(deepfry_quantum_samples_array))
    
    copy_result = ((deepfry_result - np.min(deepfry_result)) / (np.max(deepfry_result) - np.min(deepfry_result))) * (32767 + 32768) - 32768
    deepfry_quantum_samples_array = np.array(copy_result, dtype=np.int16)

newwave_name = f'neuron{time.time()}.wav'
with wave.open(newwave_name, 'w') as wav_file:
    
    wav_file.setparams((1, 2, sample_rate, length, 'NONE', 'not compressed'))
    
   
    deepfry_quantum_samples_array = np.array(copy_result, dtype=np.int16)
    print(deepfry_quantum_samples_array)
    wav_file.writeframes(np.array(deepfry_quantum_samples_array).tobytes())

display(Audio(newwave_name))

# Original Frequency VS. Brain Freezed!

In [ ]:
sample_rate, data = scipy.io.wavfile.read(newwave_name)
print(sample_rate)
# If stereo, convert to mono by averaging the two channels
if len(data.shape) == 2:
    data = data.mean(axis=1)

# Perform the Fourier Transform to get frequency content
# Use np.fft.rfft to handle real inputs and improve efficiency
freq_data = np.fft.rfft(data)

# Get the power spectrum (magnitude of the Fourier coefficients)
power_spectrum = np.abs(freq_data)

# Generate frequency axis (only for positive frequencies)
freqs = np.fft.rfftfreq(len(data), d=1/sample_rate)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(freqs, power_spectrum)
plt.title(f'Frequency Content of {filename}')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.yscale('log')  # Set the y-axis to a logarithmic scale
plt.xlim(0, sample_rate/2)  # Nyquist limit
plt.grid(True)
plt.show()

Observing the upper segment of this frequency spectrum reveals higher frequencies after ~ 6000Hz

In [ ]:
#graphing
plt.figure(figsize=(10, 6))
plt.plot(copy_result, marker='o')
plt.plot(orig_result, marker='x', label='Original Results')
plt.xlim(0, 850000)
plt.title("Array Data")
plt.xlabel("Index")
plt.ylabel("Value")
plt.grid(True)
plt.show()

At approximately 30000, supplementary frequencies have arisen, validating the distortion effect created using quantum circuits. Presented below is a comparison between the original waveform (orange) and the quantum 'brainfreezed' waveform (blue).

# Conclusion/Improvements

The results are expected because the neural data was played as staccato and therefore there was randomness before the QPAM was used. 
- In the future I will add more variational neuronal sound patterns to be run simultaneously (perhaps with Quantum Machine Learning)
- I will also neuron sounds that have higher frequencies and predict how the QPAM will amplify the frequencies.
- I will also experiment with shorter wav files to see how it affects the array data.
- I will create my own qwav files from scratch using a quantum synthesizer

# References

> https://www.blakeporterneuro.com/science/laboratory-teaching-resources/sounds-of-the-brain-neurons-and-rhythms/

> https://link.springer.com/book/10.1007/978-3-031-13909-3

> https://iccmr-quantum.github.io/

> https://github.com/erenutku97/An-Introduction-to-Quantum-Probability-Amplitude-Modulation-from-a-Compositional-Perspective

> https://google-research.github.io/seanet/brain2music/

> https://link.springer.com/chapter/10.1007/978-3-319-76054-4_13

> https://arxiv.org/pdf/2101.03887

> https://www.researchgate.net/publication/366874916_Quantum_Representations_of_Sound_from_mechanical_waves_to_quantum_circuits